In [68]:
import mwclient
import time


# site = mwclient.Site("en.wikipedia.org")

In [69]:
from textblob import TextBlob

In [70]:
site = mwclient.Site("en.wikipedia.org")

In [71]:
page= site.pages["Ethereum"]

In [72]:
revs= list(page.revisions())

In [73]:
revs

[OrderedDict([('revid', 1264362186),
              ('parentid', 1264294733),
              ('user', 'Retimuko'),
              ('timestamp',
               time.struct_time(tm_year=2024, tm_mon=12, tm_mday=21, tm_hour=18, tm_min=54, tm_sec=30, tm_wday=5, tm_yday=356, tm_isdst=-1)),
              ('comment',
               'Undid revision [[Special:Diff/1264294733|1264294733]] by [[Special:Contributions/Sharifwiki331|Sharifwiki331]] ([[User talk:Sharifwiki331|talk]]) not a reliable source')]),
 OrderedDict([('revid', 1264294733),
              ('parentid', 1263437068),
              ('minor', ''),
              ('user', 'Sharifwiki331'),
              ('timestamp',
               time.struct_time(tm_year=2024, tm_mon=12, tm_mday=21, tm_hour=11, tm_min=6, tm_sec=37, tm_wday=5, tm_yday=356, tm_isdst=-1)),
              ('comment',
               "Added reference to Coinsholder for Ethereum's use in peer-to-peer transactions.")]),
 OrderedDict([('revid', 1263437068),
              ('parent

In [74]:
revs[0]

OrderedDict([('revid', 1264362186),
             ('parentid', 1264294733),
             ('user', 'Retimuko'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=12, tm_mday=21, tm_hour=18, tm_min=54, tm_sec=30, tm_wday=5, tm_yday=356, tm_isdst=-1)),
             ('comment',
              'Undid revision [[Special:Diff/1264294733|1264294733]] by [[Special:Contributions/Sharifwiki331|Sharifwiki331]] ([[User talk:Sharifwiki331|talk]]) not a reliable source')])

In [75]:
revs = sorted(revs, key= lambda rev:rev["timestamp"])

In [76]:
revs[0]

OrderedDict([('revid', 592567939),
             ('parentid', 0),
             ('user', 'Sanpitch'),
             ('timestamp',
              time.struct_time(tm_year=2014, tm_mon=1, tm_mday=27, tm_hour=1, tm_min=53, tm_sec=45, tm_wday=0, tm_yday=27, tm_isdst=-1)),
             ('comment',
              "[[WP:AES|←]]Created page with '{{Infobox currency | image_1 =  | image_title_1 =  | image_width_1 =  | image_2 =  | image_title_2 =  | image_width_2 =  |issuing_authority = None. The Ethereum...'")])

### This Code Sample was modified and simplified to provide faster results and more acurate sentiment analysis using the TextBlob Library but the base code is used from DataQuest's Youtube where they used a pre-trained model for sentiment analysis from HuggingFace as below, I commented it out for reference.

In [77]:
from transformers import pipeline
sentiment= pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [78]:
# The output provides score and label for each element in the output list
# def find_sentiment(text):
#     sent = sentiment([text[:500]])[0]
#     score = sent["score"]
#     if sent["label"]== "NEGATIVE":
#         score = score*(-1)
#     return score 

def find_sentiment(text):
    return TextBlob(text).sentiment.polarity


In [90]:
find_sentiment("can be worse")

-0.4

In [83]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [84]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [85]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [86]:
edits_df

,edit_count,sentiment,neg_sentiment
2014-01-27,1,0.000000,0.000000
2014-02-01,1,0.000000,0.000000
2014-04-06,5,0.000000,0.000000
2014-04-09,24,0.030456,0.000000
2014-04-10,9,0.008333,0.111111
...,...,...,...
2024-10-09,1,0.000000,0.000000
2024-10-22,1,-0.100000,1.000000
2024-11-13,1,0.000000,0.000000
2024-12-16,2,0.050000,0.000000


In [91]:
edits_df.index = pd.to_datetime(edits_df.index)

In [92]:
from datetime import datetime

dates = pd.date_range(start="2014-01-27",end=datetime.today())

In [93]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [94]:
edits_df

,edit_count,sentiment,neg_sentiment
2014-01-27,1,0.0,0.0
2014-01-28,0,0.0,0.0
2014-01-29,0,0.0,0.0
2014-01-30,0,0.0,0.0
2014-01-31,0,0.0,0.0
...,...,...,...
2025-01-07,0,0.0,0.0
2025-01-08,0,0.0,0.0
2025-01-09,0,0.0,0.0
2025-01-10,0,0.0,0.0


In [95]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [96]:
rolling_edits = rolling_edits.dropna()

In [97]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2014-02-25,0.066667,0.000000,0.0
2014-02-26,0.033333,0.000000,0.0
2014-02-27,0.033333,0.000000,0.0
2014-02-28,0.033333,0.000000,0.0
2014-03-01,0.033333,0.000000,0.0
...,...,...,...
2025-01-07,0.133333,0.001667,0.0
2025-01-08,0.133333,0.001667,0.0
2025-01-09,0.133333,0.001667,0.0
2025-01-10,0.133333,0.001667,0.0


In [98]:
rolling_edits.to_csv("wikipedia_edits.csv")

In [23]:
pip install TextBlob

Note: you may need to restart the kernel to use updated packages.


In [29]:
from textblob import TextBlob

In [30]:
text= []
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [55]:
getPolarity("I am great")
# getSubjectivity("I am very confident")

0.8